#**Práctica 4a: Problemas de Clasificación Intro.**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

**Introducción**

Una de las tares típicas que tratan de resolver las técnicas de machine learning es la de clasificar datos, para esto existen infinidad de técnicas, donde la elección de una u otra dependerá del tipo y cantidad de datos que tengamos. El objetivo en este caso es entonces tomar muestras representadas con $n$ variables para asignarles alguna etiqueta, clase o variable discreta $C_k$.

$$
\begin{align}y = f(x,\theta),  y \in \{C_1,...,C_k,...,C_K-1,C_K\}\end{align}
$$


donde $y$ es la variable discreta/clase/categoría/etiqueta a predecir, que puede tomar alguno de los $K$ valores objetivo. $x$ es un vector de $n$ características/variables o features que describen una muestra.

En el caso de una clasificación que intente predecir solo 2 estados, ej. ON/OFF, SI/NO, FALLA/NO FALLA, CALOR/FRIO, etc. $K = 2$, hablaremos de una clasificación binaria. $f$ es el modelo clasificador o simplemente clasificador, y $\theta$ son los parámetros que se optimizarán durante el entrenamiento.

**En esta práctica veremos:**

1. Regresión logística
3. Evaluación del desempeño de modelos de clasificación usando diferentes métricas
4. Visualizaciones en problemas de clasificación
5. Validación cruzada para clasificación
6. Ejercicios

## **Clasificación binaria con Regresión Logística**

Crearemos un ejemplo introductorio usando una versión del dataset ``penguins_classification.csv``. Para simplificar el problema, trataremos de predecir solo la especie de 2 de los pingüinos basado en la información extraida de la medición de sus picos (culmen)

In [ ]:
import pandas as pd

penguins = pd.read_csv("https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/datasets/penguins_classification.csv")

# solo usaremos las clases Adelie y Chinstrap
penguins = penguins.set_index("Species").loc[["Adelie", "Chinstrap"]].reset_index()
culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
target_column = "Species"

In [ ]:
penguins

Visualicemos la distribución de las características para las clases:

In [ ]:
import seaborn as sns

sns.pairplot(penguins,hue = "Species",diag_kind="hist")

Notemos que tenemos un problema bastante simple de resolver. Cuando la longitud del culmen aumenta, la probabilidad de que la especie de pinguino sea Chinstrap es cercana a 1. Sin embargo, al observar la distribución de la profundidad del culmen por si sola, vemos que no es muy útil para clasificar la especie.

Para ajustar el modelo, separemos el conjunto de datos en uno de entrenamiento y otro de prueba:

In [ ]:
from sklearn.model_selection import train_test_split

penguins_train, penguins_test = train_test_split(penguins, test_size=0.2, random_state=2)

data_train = penguins_train[culmen_columns].values
data_test = penguins_test[culmen_columns].values

target_train = penguins_train[target_column]
target_test = penguins_test[target_column]

Creamos el modelo y lo ajustamos:

In [ ]:
from sklearn.linear_model import LogisticRegression # importamos el modelo de regresión logística que usaremos en esta práctica

logistic_regression = LogisticRegression(penalty="none")
logistic_regression.fit(data_train, target_train)
accuracy = logistic_regression.score(data_test, target_test)

print(f"La exactitud en el conjunto de prueba es: {accuracy:.3f}")

Creemos la matriz de confusión y otras métricas.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

LABELS = target_test.unique() # extraemos las clases únicas a predecir

y_pred_test = logistic_regression.predict(data_test)
cm = confusion_matrix(target_test,y_pred_test,labels = LABELS)
print('La matriz de confusión es: \n',cm)

Desplegamos la matriz de confusión con un mapa de intensidad:

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=LABELS)
disp.plot()
plt.title("Matriz de confusión en datos de test")
plt.show()

Otras métricas usando `classification_report`:

In [ ]:
print(classification_report(target_test, y_pred_test, target_names=LABELS))

Grafiquemos la frontera de decisión generada con el modelo, esto lo podemos hacer con la clase `DecisionBoundaryDisplay` en un plano 2D yaque estamos trabajando solo con dos features de los datos, para mas variables sería complicado, a menos que apliquemos alguna técnica de reducción de dimensionalidad previamente para simplificar el problema.

In [ ]:
#Nota: la función DecisionBoundaryDisplay solo trabajará en versiones iguales o superiores a 1.1 de sci-kit learn,
# en colab, no permite actualizar a una de estas versiones por la versión de python (3.7) que maneja, pero dejaré
# el código para que puedan ejecturalo en en su versión de jupyter o código en sus PCs locales
import sklearn
sklearn_version = sklearn.__version__

print(sklearn_version)

In [ ]:
# versión de python en colab actual
!python --version

In [ ]:
#Ejecutar solo si se cumple el requisito de la versión de sklearn
import seaborn as sns
from sklearn.inspection import DecisionBoundaryDisplay

DecisionBoundaryDisplay.from_estimator(logistic_regression, data_test, response_method="predict", cmap="RdBu_r", alpha=0.5)
sns.scatterplot(data=data_test, x=culmen_columns[0], y=culmen_columns[1],
                hue=target_column,
                palette=["tab:red", "tab:blue"])

_ = plt.title("Frontera de decisión del modelo entrenado\n Regresión logística en los datos de prueba")
plt.show()

Gráfico de frontera de decisión usando [utilidades](https://github.com/amueller/mglearn) desarrolladas por [A. Muller](https://amueller.github.io/) (uno de los creadores de sci-kit learn, todos los créditos a él), acá definimos manualmente las funciones de esa librería yaque presenta problemas de importación, ejecutar la siguiente porción de código:

In [ ]:
#@title Ejecutar funciones auxiliares
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, colorConverter, LinearSegmentedColormap
import matplotlib as mpl

cm_cycle = ListedColormap(['#0000aa', '#ff5050', '#50ff50', '#9040a0', '#fff000'])
cm3 = ListedColormap(['#0000aa', '#ff2020', '#50ff50'])
cm2 = ListedColormap(['#0000aa', '#ff2020'])

# create a smooth transition from the first to to the second color of cm3
# similar to RdBu but with our red and blue, also not going through white,
# which is really bad for greyscale

cdict = {'red': [(0.0, 0.0, cm2(0)[0]),
                 (1.0, cm2(1)[0], 1.0)],

         'green': [(0.0, 0.0, cm2(0)[1]),
                   (1.0, cm2(1)[1], 1.0)],

         'blue': [(0.0, 0.0, cm2(0)[2]),
                  (1.0, cm2(1)[2], 1.0)]}

ReBl = LinearSegmentedColormap("ReBl", cdict)


def discrete_scatter(x1, x2, y=None, markers=None, s=10, ax=None,
                     labels=None, padding=.2, alpha=1, c=None, markeredgewidth=None):
    """Adaption of matplotlib.pyplot.scatter to plot classes or clusters.
    Parameters
    ----------
    x1 : nd-array
        input data, first axis
    x2 : nd-array
        input data, second axis
    y : nd-array
        input data, discrete labels
    cmap : colormap
        Colormap to use.
    markers : list of string
        List of markers to use, or None (which defaults to 'o').
    s : int or float
        Size of the marker
    padding : float
        Fraction of the dataset range to use for padding the axes.
    alpha : float
        Alpha value for all points.
    """
    if ax is None:
        ax = plt.gca()

    if y is None:
        y = np.zeros(len(x1))

    unique_y = np.unique(y)

    if markers is None:
        markers = ['o', '^', 'v', 'D', 's', '*', 'p', 'h', 'H', '8', '<', '>'] * 10

    if len(markers) == 1:
        markers = markers * len(unique_y)

    if labels is None:
        labels = unique_y

    # lines in the matplotlib sense, not actual lines
    lines = []

    current_cycler = mpl.rcParams['axes.prop_cycle']

    for i, (yy, cycle) in enumerate(zip(unique_y, current_cycler())):
        mask = y == yy
        # if c is none, use color cycle
        if c is None:
            color = cycle['color']
        elif len(c) > 1:
            color = c[i]
        else:
            color = c
        # use light edge for dark markers
        if np.mean(colorConverter.to_rgb(color)) < .4:
            markeredgecolor = "grey"
        else:
            markeredgecolor = "black"

        lines.append(ax.plot(x1[mask], x2[mask], markers[i], markersize=s,
                             label=labels[i], alpha=alpha, c=color,
                             markeredgewidth=markeredgewidth,
                             markeredgecolor=markeredgecolor)[0])

    if padding != 0:
        pad1 = x1.std() * padding
        pad2 = x2.std() * padding
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        ax.set_xlim(min(x1.min() - pad1, xlim[0]), max(x1.max() + pad1, xlim[1]))
        ax.set_ylim(min(x2.min() - pad2, ylim[0]), max(x2.max() + pad2, ylim[1]))

    return lines

def plot_2d_classification(classifier, X, fill=False, ax=None, eps=None,
                           alpha=1, cm=cm3):
    # multiclass
    if eps is None:
        eps = X.std() / 2.

    if ax is None:
        ax = plt.gca()

    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 1000)
    yy = np.linspace(y_min, y_max, 1000)

    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    decision_values = classifier.predict(X_grid)
    ax.imshow(decision_values.reshape(X1.shape), extent=(x_min, x_max,
                                                         y_min, y_max),
              aspect='auto', origin='lower', alpha=alpha, cmap=cm)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

def plot_2d_separator(classifier, X, fill=False, ax=None, eps=None, alpha=1,
                      cm=cm2, linewidth=None, threshold=None,
                      linestyle="solid"):
    # binary?
    if eps is None:
        eps = X.std() / 2.

    if ax is None:
        ax = plt.gca()

    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 1000)
    yy = np.linspace(y_min, y_max, 1000)

    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    try:
        decision_values = classifier.decision_function(X_grid)
        levels = [0] if threshold is None else [threshold]
        fill_levels = [decision_values.min()] + levels + [
            decision_values.max()]
    except AttributeError:
        # no decision_function
        decision_values = classifier.predict_proba(X_grid)[:, 1]
        levels = [.5] if threshold is None else [threshold]
        fill_levels = [0] + levels + [1]
    if fill:
        ax.contourf(X1, X2, decision_values.reshape(X1.shape),
                    levels=fill_levels, alpha=alpha, cmap=cm)
    else:
        ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels,
                   colors="black", alpha=alpha, linewidths=linewidth,
                   linestyles=linestyle, zorder=5)

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(7,5))

plot_2d_separator(logistic_regression, data_test, fill=True, eps = 0.5, alpha=.4)
discrete_scatter(data_test[:,0], data_test[:,1], target_test)


plt.title("Frontera de decisión del modelo entrenado\n Regresión logística en los datos de prueba")
plt.xlabel(culmen_columns[0])
plt.ylabel(culmen_columns[1])
plt.legend(LABELS)

# ajuste ejes (opcional)
plt.xticks(np.arange(data_test[:,0].min(),data_test[:,0].max(), step=5))
plt.yticks(np.arange(data_test[:,1].min(),data_test[:,1].max()))
plt.axis("tight")

plt.show()

Luego, vemos que la función de decisión es representada por una linea seaparando las dos clases. En este caso, se usó una combinación de ambos features para graficar la frontera. Veamos ahora los valores de los coeficientes estimados para cada variable predictora para tener una idea de la importancia que le atribuye el modelo a cada una:

In [ ]:
logistic_regression.coef_

In [ ]:
coefs = logistic_regression.coef_[0]
weights = pd.Series(coefs, index=culmen_columns)

In [ ]:
weights.plot.barh()
_ = plt.title("Pesos de la regresión logística")

En este caso, ambos coeficientes son no nulos. Si alguno de ellos hubiese sido cero, la frontera de decisión hubise sido o una recta horizontal o vertical.

Más aún, el intercepto tambien es distinto de cero, lo que significa que la decisión no pasa por las coordenadas (0,0), este tiene un valor de:

In [ ]:
logistic_regression.intercept_

Para extraer alguna información acerca de la importancia de los coeficientes anteriores respecto a las variables que acompañan, las variables debieran estar en la misma escala o rango de valores, en el caso anterior no es así. La interpretación puede ser errónea si concluimos que la variable `Culmen Depth` es más importante para lograr la predicción, cuando ya vimos de los histogramas que la variable `Culmen Length` es la que facilita dicha discriminación entre especies. Resolvamos esto entrenando un modelo pero con las variables estandarizadas, es decir, cada columna de la matriz de datos tendrá media  = 0 y desv. estándar = 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Preprocesamiento de los datos (columnas), en este caso los estandaricemos, i.e. los dejaremos con media 0 y desviación estándar 1
scaler = StandardScaler()
X_data = scaler.fit_transform(penguins[culmen_columns])
# Modelo
LogReg_ss  = LogisticRegression()# generamos una instancia del modelo

LogReg_ss.fit(X_data, penguins[target_column].values)# entrenamos en todos los datos, solo para evaluar los coeficientes
coefs = LogReg_ss.coef_[0]
weights_ss = pd.Series(coefs, index=culmen_columns)# coeficientes nuevo modelo

# gráfico de barras con la amplitud de los pesos cuando el modelo se entrena con las variables estandarizadas
weights_ss.plot.barh()
_ = plt.title("Pesos de la regresión logística")


A partir del gráfico, vemos que el resultado es mas coherente respecto a los histogramas iniciales, donde la variable `Culmen Length`está aportando mayor información al modelo para realizar la discriminación de especies. Hacer este pre-procesamiento para todos los modelos donde la interpretación de coeficientes u optimizados sea necesaria.

Finalmente, si volvemos al primer paso de división de los datos entre un conjunto de entrenamiento y otro de prueba, qué sucedería si tomamos otra división? se mantendrían las métricas? esto lo podemos verificar cambiando el valor de `random_state=2` a otro número, por ejemplo: `random_state=5`. Verificar.

Cómo podemos entregar una métrica de desempeño más realista del modelo?

R: **Usando validación cruzada.**

### **Validación cruzada para mejorar el reporte de métricas de desempeño**

**Recordemos**: La validación cruzada nos permitirá estimar la robustez de un modelo predictivo al repetir el procedimiento de división del conjunto de datos en conjuntos de *train* y *test*. Este procedimiento nos dará varios valores de métricas de desempeño en entrenamiento y prueba y por consiguiente alguna **estimación de la variabilidad del desempeño de generalización del modelo**.

Aquí usaremos la estrategia de *K-fold cross validation*, que, como muestra la imagen de más abajo, subdivide el conjunto de datos en K grupos, probando el modelo K veces en una porción diferente de los datos.


<center><img src=https://scikit-learn.org/stable/_images/grid_search_cross_validation.png width="650" ></center>


[fuente](https://scikit-learn.org/stable/modules/cross_validation.html)

En el siguiente ejemplo no usaremos un conjunto de prueba extra, solo reportaremos las métricas en los grupos divididos (folds) sacados del conjunto de datos.



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold


SCORING = ['accuracy','precision_macro'] # métricas de desempeño que nos interesará evaluar, podemos agregar si queremos
                                         # usar sklearn.metrics.SCORERS.keys() para ver las métricas disponibles

logistic_regresion_cv = LogisticRegression(penalty="none")# generamos una instancia del modelo

CV         = KFold(n_splits = 10, shuffle = True, random_state = 1)# definimos el tipo de estrategia de validación cruzada, KFold en este caso con 10 folds y con reordanmiento aleatorio de los datos originales
cv_results = cross_validate(logistic_regresion_cv, penguins[culmen_columns].values, penguins[target_column].values, cv = CV,
                            scoring=SCORING,
                            return_train_score=True)# aplicamos la estrategia


In [ ]:
# visualicemos los resultados
import pandas as pd
cv_results = pd.DataFrame(cv_results)
cv_results

Obtenemos la información de las métricas de interés y tiempos para ajustar y predecir en cada iteración de validación cruzada. También tenemos el score para prueba, el cual corresponde a un error de prueba en cada una de las divisiones.

In [ ]:
len(cv_results)

En total tenemos 10 entradas en el dataframe resultante dado que definimos 10 divisiones del conjunto de datos. Por consiguiente, podemos estimar métricas promedio y su desviación estándar para tener un indicio de su variabilidad.

In [ ]:
print(f"El promedio del accuracy en los conjuntos de prueba de validación cruzada es: "
      f"{cv_results['test_accuracy'].mean():.2f}")

In [ ]:
print(f"La desviación estándar del accuracy usando validación cruzada es:"
      f"{cv_results['test_accuracy'].std():.2f}")

Notamos que el accuracy es mucho más grande que la desviación estándar reportada, esto nos dice que la variabilidad del accuracy es aproximadamente del 3%.

**Finalmente, qué modelo usamos para predecir la etiqueta de nuevas muestras?** lo que hacemos es entrenar el modelo, en este caso el de regresión logística, esta vez en todos los datos disponibles, pero considerando como métricas de desempeño las obtenidas con la estrategia anterior, esto nos da una indicación mas realista sobre como se debiera comportar el modelo en nuevos datos.

### **Ejercicio complementario**

En este problema, resolver el mismo ejercicio anterior, pero ahora considerar el conjunto de datos con las tres especies de pingüinos y todas las variables.

**Nota:** Reportar al menos una métrica de desempeño usando validación cruzada y la matriz de confusión usando todo el conjunto de datos.

**Solución:**

In [ ]:
# Cargamos todos los datos
import pandas as pd
penguins = pd.read_csv("https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/datasets/penguins_classification.csv")

data    = penguins[["Culmen Length (mm)", "Culmen Depth (mm)"]].values
target  = penguins["Species"].values
LABELS  = penguins["Species"].unique()

In [ ]:
# Aplicamos la estrategia de validación cruzada y entrenamos el modelo de regresión logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold


lr_penguins = LogisticRegression(penalty="none")# generamos una instancia del modelo

CV         = KFold(n_splits = 10, shuffle = True, random_state = 1)
cv_results = cross_validate(lr_penguins, data, target, cv = CV, scoring='accuracy')# aplicamos la estrategia

print(f"El promedio del accuracy (y su desviación estándar) en los conjuntos de prueba de validación cruzada es: "
      f"{cv_results['test_score'].mean():.2f} ({cv_results['test_score'].std():.2f})")

Ojo, ahora como el único score que definí arriba es accuracy, se accede como `test_score` en `cv_results`. Esto se puede verificar fácilmente inspeccionado las variables generadas. Entrenemos ahora el modelo en todos los datos disponibles y despleguemos la matriz de confusión:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

lr_penguins.fit(data,target)#entrenamos el modelo en todos los datos disponibles
cm = confusion_matrix(target,lr_penguins.predict(data),labels = LABELS)# calculamos matriz de confusión

disp = ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=LABELS)# desplegamos matriz de confusión
disp.plot()
plt.title("Matriz de confusión conjunto datos completo")
plt.show()

## **Referencias**

1. Cusro sci-kit learn MOOC Inria: https://inria.github.io/scikit-learn-mooc/index.html
2. Sci-kit learn documentación:

* Validación cruzada: [enlace](https://scikit-learn.org/stable/modules/cross_validation.html)
* Regresión Logística: [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
* Matriz de confusión: [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)